In [1]:
import numpy as np
import pandas as pd

## READ THE DATA

In [2]:
fileName = 'input.txt'

with open(fileName) as openFile:
    seatArray = [[character for character in line.rstrip()] for line in openFile]

seatData = pd.DataFrame(seatArray)
seatData.head()

,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
0,L,L,L,L,L,L,.,L,L,L,...,.,L,L,L,L,L,L,L,L,L
1,L,L,L,L,L,L,.,L,L,L,...,L,L,L,L,L,L,L,.,L,L
2,L,L,L,L,L,L,L,L,L,L,...,.,L,L,.,L,L,L,L,L,L
3,L,L,L,L,L,L,L,L,L,L,...,.,L,L,L,L,L,L,L,L,L
4,L,L,L,L,L,.,.,L,L,L,...,.,L,L,L,L,L,L,L,L,L


## PART 1

In [3]:
def CheckIfSeatTaken(dataFrame, seatRow, seatColumn):
    dataFrameLimit = dataFrame.shape[0] - 1
    borderCondition = ((seatRow < 0) or (seatRow > dataFrameLimit) or (seatColumn < 0) or (seatColumn > dataFrameLimit))
    
    if borderCondition:
        return False
    elif dataFrame.loc[seatRow, seatColumn] == '#':
        return True
    else:
        return False

def AdjacentSeatsTaken(dataFrame, seatRow, seatColumn):
    adjacentSeatsTaken = 0
    
    for row in range(seatRow-1, seatRow+2):
        for column in range(seatColumn-1, seatColumn+2):
            if (row, column) == (seatRow, seatColumn):
                continue
            adjacentSeatsTaken += CheckIfSeatTaken(dataFrame, row, column)

    return adjacentSeatsTaken
    

In [4]:
def AdvanceOneStep(dataFrame):
    dataFrameNextStep = dataFrame.copy()
    
    for row in range(dataFrame.shape[0]):
        for column in range(dataFrame.shape[1]):
            if dataFrame.loc[row, column] == 'L':
                if AdjacentSeatsTaken(dataFrame, row, column) == 0:
                    dataFrameNextStep.loc[row, column] = '#'
            elif dataFrame.loc[row, column] == '#':
                if AdjacentSeatsTaken(dataFrame, row, column) >= 4:
                    dataFrameNextStep.loc[row, column] = 'L'
        
    return dataFrameNextStep

In [5]:
previousDataFrame = seatData.copy()
#step = 1

while True:
    #if step % 10 == 0:
    #    print("Processing step", step)
    #step += 1
    nextDataFrame = AdvanceOneStep(previousDataFrame)
    if nextDataFrame.equals(previousDataFrame):
        break
    else:
        previousDataFrame = nextDataFrame.copy()
        
print(previousDataFrame)
print('\n')
print("Number of taken seats:", previousDataFrame.isin(['#']).to_numpy().sum())

   0  1  2  3  4  5  6  7  8  9   ... 88 89 90 91 92 93 94 95 96 97
0   #  L  #  #  L  #  .  #  L  #  ...  .  #  L  #  L  #  L  #  L  #
1   L  L  L  L  L  L  .  L  L  L  ...  L  L  L  L  L  L  L  .  L  L
2   #  L  #  L  #  L  #  L  #  L  ...  .  #  L  .  L  #  L  #  L  #
3   L  L  L  L  L  L  L  L  L  L  ...  .  L  L  #  L  L  L  L  L  L
4   #  L  #  L  #  .  .  #  #  L  ...  .  #  L  #  L  #  L  #  L  #
.. .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. .. ..
93  #  L  #  L  #  L  #  L  #  L  ...  #  L  #  L  #  .  #  #  L  #
94  L  L  L  L  L  .  L  L  L  L  ...  .  L  L  L  L  L  L  L  L  L
95  #  L  #  L  #  #  .  #  L  #  ...  .  #  L  #  L  #  L  #  L  #
96  L  L  L  L  L  L  .  L  L  L  ...  L  L  L  L  L  L  L  L  L  .
97  #  L  #  L  #  L  #  L  #  L  ...  .  #  L  #  L  #  L  #  L  #

[98 rows x 98 columns]


Number of taken seats: 2470


## PART 2

In [6]:
def FindFirstSeat(dataFrame, startingRow, startingColumn, direction):
    if direction not in ['u', 'd', 'l', 'r', 'ur', 'ul', 'dr', 'dl']:
        print("ERROR:", direction, "is not a valid direction.")
        return False
    
    rowIncrement = 0
    columnIncrement = 0
    
    if direction in ['u', 'ur', 'ul']:
        rowIncrement -= 1
    if direction in ['d', 'dr', 'dl']:
        rowIncrement += 1
    if direction in ['l', 'ul', 'dl']:
        columnIncrement -= 1
    if direction in ['r', 'ur', 'dr']:
        columnIncrement += 1
    
    row = startingRow
    column = startingColumn
    dataFrameLimit = dataFrame.shape[0] - 1
    
    while True:
        row += rowIncrement
        column += columnIncrement
        borderCondition = ((row < 0) or (row > dataFrameLimit) or (column < 0) or (column > dataFrameLimit))
        if borderCondition:
            break
        
        if dataFrame.loc[row, column] != '.':
            break
            
    return (row, column)

def CountTakenSeatsInLineOfSight(dataFrame, row, column):
    countOfTakenSeats = 0
    
    for direction in ['u', 'd', 'l', 'r', 'ur', 'ul', 'dr', 'dl']:
        countOfTakenSeats += CheckIfSeatTaken(dataFrame, *FindFirstSeat(dataFrame,row,column,direction))
    
    return countOfTakenSeats

def AdvanceOneStep_Part2(dataFrame):
    dataFrameNextStep = dataFrame.copy()
    
    for row in range(dataFrame.shape[0]):
        for column in range(dataFrame.shape[1]):
            if dataFrame.loc[row, column] == 'L':
                if CountTakenSeatsInLineOfSight(dataFrame, row, column) == 0:
                    dataFrameNextStep.loc[row, column] = '#'
            elif dataFrame.loc[row, column] == '#':
                if CountTakenSeatsInLineOfSight(dataFrame, row, column) >= 5:
                    dataFrameNextStep.loc[row, column] = 'L'
        
    return dataFrameNextStep

In [7]:
previousDataFrame = seatData.copy()
#step = 1

while True:
    #if step % 10 == 0:
    #    print("Processing step", step)
    #step += 1
    nextDataFrame = AdvanceOneStep_Part2(previousDataFrame)
    if nextDataFrame.equals(previousDataFrame):
        break
    else:
        previousDataFrame = nextDataFrame.copy()
        
print(previousDataFrame)
print('\n')
print("Number of taken seats:", previousDataFrame.isin(['#']).to_numpy().sum())

   0  1  2  3  4  5  6  7  8  9   ... 88 89 90 91 92 93 94 95 96 97
0   #  L  #  L  #  L  .  #  L  #  ...  .  #  L  #  L  #  L  #  L  #
1   L  L  L  L  L  L  .  L  L  L  ...  L  L  L  L  L  L  L  .  L  L
2   #  L  #  L  #  L  #  L  #  L  ...  .  #  L  .  #  L  #  L  L  #
3   L  L  L  L  L  L  L  L  L  L  ...  .  L  L  L  L  L  L  #  L  L
4   #  L  #  L  #  .  .  L  #  L  ...  .  #  L  #  L  #  L  L  L  #
.. .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. .. ..
93  #  L  #  L  #  #  L  #  L  #  ...  #  L  #  L  #  .  L  #  L  L
94  L  L  L  L  L  .  L  L  L  L  ...  .  L  L  L  L  L  L  L  L  #
95  #  L  #  L  #  L  .  #  L  #  ...  .  L  #  L  #  L  #  L  #  L
96  L  L  L  L  L  L  .  L  L  L  ...  L  L  L  L  L  L  L  L  L  .
97  #  L  #  L  #  L  #  L  #  L  ...  .  L  #  L  #  L  #  L  #  #

[98 rows x 98 columns]


Number of taken seats: 2259
